In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime
from pandas.tseries.offsets import BDay
from dateutil.relativedelta import relativedelta
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import mean_squared_error 
from geneticalgorithm import geneticalgorithm as ga
import pickle

In [17]:
mean = data2.groupby(data2.afterdate).conm.nunique().mean()
plt.plot(data2.groupby(data2.afterdate).conm.nunique())
plt.title('Number of companies per quarter')
plt.axhline(y = mean, color='r', linestyle=' — ')

NameError: name 'data2' is not defined

In [13]:
def model_funk(svrdt, trainStart, trainEnd, kernelValue, cValue, gammaValue, epsilonValue, compToChoose):   
    
    #handle with infinities
    svrdt = svrdt[~svrdt.isin([np.nan, np.inf, -np.inf]).any(1)]
    
    #turn below three lines into comment if you choose not to create company dummies? create company dummies
    
    dummy = pd.get_dummies(svrdt['conm'])
    svrdt = pd.concat([svrdt,dummy], axis = 1)
    
    #dividing data into train and test sets depending on trainStart and trainEnd dates
    svrdt['afterdate'] = pd.to_datetime(svrdt['afterdate'].astype(str), format='%Y-%m-%d')
    svrdt = svrdt.sort_values('afterdate')
    train = svrdt[(svrdt['year'] < trainEnd) & (svrdt['year'] >= trainStart)]
    test = svrdt[svrdt['year'] == trainEnd]
    trainIx = svrdt[(svrdt['year'] < trainEnd) & (svrdt['year'] >= 2010)].index
    testIx = svrdt[svrdt['year'] >= trainEnd].index
    
    Stt = train
    #choose explanatory variables for train
    S  = train.drop(['afterdate','conm','futureQDate','qReturn'], axis=1)
    #choose dependent variable for train
    t = train[['qReturn']]
    SSS = test
    #choose explanatory variables for test
    SS = test.drop(['afterdate','conm','futureQDate','qReturn'], axis=1)
    #choose dependent variable for test
    tt = test[['qReturn']]
#feature scaling for train
    sc_S = StandardScaler()
    sc_t = StandardScaler()
    S2 = sc_S.fit_transform(S)
    t2 = sc_t.fit_transform(t)
#feature scaling for test
    scc_S = StandardScaler()
    scc_t = StandardScaler()
    SS2 = scc_S.fit_transform(SS)
    tt2 = scc_t.fit_transform(tt)
#create SVR model
    regressor = SVR(kernel = kernelValue ,C= cValue, gamma = gammaValue, epsilon = epsilonValue)
    #fit the model to train data
    regressor.fit(S2, t2)
    #predict test data results 
    preds = regressor.predict(SS2)
    SSS['preds'] = preds
#choose best N companies for the portfolio for each quarter in the test set
    top10 = SSS.groupby('afterdate').apply(lambda x: x.nlargest(compToChoose, 'preds')).reset_index(drop=True) 
    a = top10.groupby('afterdate')['qReturn'].mean()
    a = a+1
    
    #create portfolio return for the test year
    a.index = pd.to_datetime(a.index)
    a = pd.DataFrame(a)
    a['cumulativeReturn'] = 1
    a = a.rename(columns={"qReturn": "portfolioReturn"})
    x = 100
#create cumulative return column
    for i in range(len(a)):
        a.iloc[i, 1] = a.iloc[i, 0]*x
        x = a.iloc[i, 1]
    a['date'] = a.index
    return a

In [14]:

def modelAll1(data2, kernelValue, cValue, gammaValue, epsilon, compToChoose):  
    #create return predictions for each year in the between 2013 and 2020
    p13 = model_funk(data2, 2010, 2013, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p14 = model_funk(data2, 2011, 2014, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p15 = model_funk(data2, 2012, 2015, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p16 = model_funk(data2, 2013, 2016, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p17 = model_funk(data2, 2014, 2017, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p18 = model_funk(data2, 2015, 2018, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    p19 = model_funk(data2, 2016, 2019, kernelValue, cValue, gammaValue, epsilon, compToChoose)
    
    #concatanate all the predictions
    pAll = pd.concat([p13,p14,p15,p16,p17,p18,p19])
    x = 100
#create cumulative return column
    for i in range(len(pAll)):
        pAll.iloc[i, 1] = pAll.iloc[i, 0]*x
        x = pAll.iloc[i, 1]
    
    return pAll

In [15]:
#define kernel, c, gamma, epsilon and number of company values for parameter search space 
cVals =  [0.001, 0.1, 1, 5, 10]
gammaVals =  [10, 1, 0.01, 'scale', 'auto_deprecated']
kernelVals =['linear','rbf', 'sigmoid']
epsilonVals = [0.001,0.1,1, 10]
numOfComps = [10]
#creating an empty Dataframe with column names only
solutions = pd.DataFrame(columns=['return' ,'kernel', 'C', 'gamma', '# of comps'])
#test different SVR parameters in the search space
for t in kernelVals: 
    for i in cVals:
        for j in gammaVals:
            for e in epsilonVals:
                for n in numOfComps:
                    ret = modelAll1(data2, t, i, j, e, n)
                    retVal = ret['cumulativeReturn'].iloc[-1]
                    solutions = solutions.append({'return': retVal, 'kernel': t, 'C': i, 'gamma': j, 'epsilon': e, '# of comps': n}, ignore_index=True)
                    #see where the for loop is. mine took about 50 minutes ((:
                    print(t)
                    print(i)
                    print(j)
                    print(e)
                    print(n)

NameError: name 'data2' is not defined